### Descripción

Contexto de negocio.

El cáncer de próstata es uno de los tipos de cáncer más comunes en hombres. La detección temprana es crucial para mejorar las tasas de supervivencia. La prueba de antígeno prostático específico (PSA) puede ayudar a detectar el cáncer de próstata en etapas tempranas, cuando es más tratable. Sin embargo, el PSA no es específico para el cáncer de próstata y puede estar elevado en otras condiciones como prostatitis o hiperplasia prostática benigna (HPB).

El antígeno prostático específico (PSA) es una proteína producida por células normales y malignas de la glándula prostática. La prueba del PSA mide el nivel de esta proteína en la sangre y es uno de los métodos más utilizados para el tamizaje del cáncer de próstata.

La EPS SaludPorTi, está interesado en priorizar la toma de está prueba, aumentando la demanda y detención temprana del Cáncer de Próstata.

Problema de negocio
La empresa ha decidido contratarlos para que construyan un modelo predictivo que permita estimar la probabilidad de que un usuario entre 48 y 60 años de edad presente resultados anormales de PSA.


Contexto analítico

Se espera que entrene diferentes familias de modelos predictivos de clasificación (SVC con diferentes kernels, Redes Neuronales poco profundas), precedidos por diferentes procesos de transformación (normalizaciones, imputación, feature engineering, dummificación, PCA, selección de features).


La evaluación de la calidad de los flujos de modelos predictivos se debe estimar utilizando la métrica de ROC_AUC.

Expliquen sus ideas, el por qué realizan las acciones, y comenten los resultados obtenidos; se espera mucho más que unos bloques de código.
La toma de decisiones sobre los datos se debe hacer considerando el contexto del problema y de los datos, no se puede ver todo solamente desde los ojos de los datos, sino también considerar el negocio.
Un Científico de Datos debe poder comunicar los puntos importantes de su trabajo en un lenguaje universal para todos los públicos.
Todo esto se considerará en la nota.

### Cita

@misc{fa-ii-2024-ii-flujos-de-modelos-tradicionales,
    author = {Daniel Osorio, JavierDiaz},
    title = {FA II 2024-II: flujos de modelos tradicionales},
    publisher = {Kaggle},
    year = {2024},
    url = {https://kaggle.com/competitions/fa-ii-2024-ii-flujos-de-modelos-tradicionales}
}

### 1. Librerías a básicas utilizar

In [1]:
# Importar librerías

# Manejo de analísis de datos a través de dataframes (data tabular)
import pandas as pd
# Manipulación de arreglos y análisis numérico
import numpy as np
# Visualización de datos
import seaborn as sns
# Visualización de datos
import matplotlib.pyplot as plt
%matplotlib inline
# Librería para el manejo de expresiones regulares
import re

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")


### 2. Lectura de datos

In [2]:
df = pd.read_parquet(r"https://github.com/alfa7g7/Fundamentos-analitica-II/raw/refs/heads/main/UNIDAD%20II/Data/df_train.parquet")
print(df.shape)
df.head()

(23494, 46)


,MEDICAMENTOS,MEDICINA ESPECIALIZADA,MEDICINA GENERAL,Cant_gr_flia,Cant_riesgos_flia_mean,min_Tiempo_CP_Fliar,Cant_Fliar_CP,psa_max_gr_flia,psa_min_gr_flia,Cant_Fliar_riesgos,...,CORONARIOS_FAMILIAR,CEREBRAL,CEREBRAL_FAMILIAR,ENFERMEDAD_RENAL,ENFERMEDAD_RENAL_FAMILIAR,OTROS_ANTECEDENTES_VASCULARES,Target,Pendiente,Intercepto,Promedio_costo
ID,,,,,,,,,,,,,,,,,,,,,
16484,12.0,3.0,3.0,2,0.0,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,58373.057143,-110021.200000,1.885690e+05
11528,6.0,1.0,1.0,2,0.5,NaN,0.0,NaN,NaN,1.0,...,0,0,0,0,0,0,0,19618.942857,-36176.133333,9.747050e+04
26630,1.0,0.0,2.0,2,0.0,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,1,398534.000000,-626628.000000,2.304723e+06
12586,6.0,0.0,2.0,2,0.0,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,1292.600000,13410.733333,5.380450e+04
11225,3.0,0.0,2.0,1,1.0,NaN,0.0,NaN,NaN,1.0,...,2,2,2,2,2,2,0,13728.714286,-27375.666667,4.134967e+04


In [3]:
df1 = pd.read_parquet(r"https://github.com/alfa7g7/Fundamentos-analitica-II/raw/refs/heads/main/UNIDAD%20II/Data/df_test.parquet")
print(df1.shape)
df1.head()

(10069, 45)


,MEDICAMENTOS,MEDICINA ESPECIALIZADA,MEDICINA GENERAL,Cant_gr_flia,Cant_riesgos_flia_mean,min_Tiempo_CP_Fliar,Cant_Fliar_CP,psa_max_gr_flia,psa_min_gr_flia,Cant_Fliar_riesgos,...,CORONARIOS,CORONARIOS_FAMILIAR,CEREBRAL,CEREBRAL_FAMILIAR,ENFERMEDAD_RENAL,ENFERMEDAD_RENAL_FAMILIAR,OTROS_ANTECEDENTES_VASCULARES,Pendiente,Intercepto,Promedio_costo
ID,,,,,,,,,,,,,,,,,,,,,
12596,2.0,4.0,2.0,2,0.000000,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,5.421180e+06,-1.256617e+07,9.611937e+06
29326,4.0,0.0,1.0,1,0.000000,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,-9.504343e+03,5.997787e+04,8.013800e+04
24338,0.0,0.0,1.0,2,0.000000,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,2.477143e+03,-5.780000e+03,1.734000e+04
11167,1.0,0.0,0.0,1,0.000000,NaN,0.0,NaN,NaN,0.0,...,1,0,1,0,1,0,0,3.597143e+03,-8.393333e+03,2.518000e+04
22074,36.0,0.0,1.0,3,0.666667,NaN,0.0,NaN,NaN,1.0,...,0,0,0,0,0,0,0,-9.042514e+03,6.309881e+05,5.993393e+05


In [28]:
#pasar a csv
#df.to_csv(r"C:\Users\alfa7\OneDrive\Documentos\ICESI\MAESTRIA CIENCIA DE DATOS\2do semestre\Fundamentos de analitica II\Unidad II\Proyecto PSA\Data\df_train.csv")
#df1.to_csv(r"C:\Users\alfa7\OneDrive\Documentos\ICESI\MAESTRIA CIENCIA DE DATOS\2do semestre\Fundamentos de analitica II\Unidad II\Proyecto PSA\Data\df_test.csv")

### 3. Análisis Explotarorio de Datos

- Limpieza de datos
- Exploración de datos
- Ingeniería de datos

### 3.1. Limpieza de datos

Para realizar una apropiada limpieza de datos se deben entender cada una de las variables presentes dentro del conjunto de datos.

Elementos a tener en cuenta:
- El significado y tipo (e.g. nominal/ordinal/intervalo/ratio) de cada una de las variables
- Identificación de valores faltantes y en caso de haberlos plantear las opciones para tratarlos
- Presencia de datos atípicos y en caso de haberlos identificar la manera de tratarlos

In [4]:
df.columns

Index(['MEDICAMENTOS', 'MEDICINA ESPECIALIZADA', 'MEDICINA GENERAL',
       'Cant_gr_flia', 'Cant_riesgos_flia_mean', 'min_Tiempo_CP_Fliar',
       'Cant_Fliar_CP', 'psa_max_gr_flia', 'psa_min_gr_flia',
       'Cant_Fliar_riesgos', 'cantidad_serv_flia', 'Pendiente_flia',
       'Intercepto_flia', 'Promedio_costo_flia', 'ESTADO_CIVI', 'PROGRAMA',
       'estrato', 'IMC', 'AGRUPACION_SISTOLICA', 'AGRUPACION_DIASTOLICA',
       'parentesco', 'EDAD', 'RIESGOS', 'CANTIDAD_SERVICIOS',
       'PERDIDA_DE_PESO', 'TIEMPO_ULTIMA_CITA', 'conteo_dx_diferentes',
       'TIEMPO_AFILIACION', 'CANCER_MAMA_FAMILIAR', 'CANCER_OTRO_SITIO',
       'CANCER_OTRO_SITIO_FAMILIAR', 'HIPERTENSION', 'HIPERTENSION_FAMILIAR',
       'DIABETES', 'DIABETES_FAMILIAR', 'CORONARIOS', 'CORONARIOS_FAMILIAR',
       'CEREBRAL', 'CEREBRAL_FAMILIAR', 'ENFERMEDAD_RENAL',
       'ENFERMEDAD_RENAL_FAMILIAR', 'OTROS_ANTECEDENTES_VASCULARES', 'Target',
       'Pendiente', 'Intercepto', 'Promedio_costo'],
      dtype='object')

In [5]:
df1.columns

Index(['MEDICAMENTOS', 'MEDICINA ESPECIALIZADA', 'MEDICINA GENERAL',
       'Cant_gr_flia', 'Cant_riesgos_flia_mean', 'min_Tiempo_CP_Fliar',
       'Cant_Fliar_CP', 'psa_max_gr_flia', 'psa_min_gr_flia',
       'Cant_Fliar_riesgos', 'cantidad_serv_flia', 'Pendiente_flia',
       'Intercepto_flia', 'Promedio_costo_flia', 'ESTADO_CIVI', 'PROGRAMA',
       'estrato', 'IMC', 'AGRUPACION_SISTOLICA', 'AGRUPACION_DIASTOLICA',
       'parentesco', 'EDAD', 'RIESGOS', 'CANTIDAD_SERVICIOS',
       'PERDIDA_DE_PESO', 'TIEMPO_ULTIMA_CITA', 'conteo_dx_diferentes',
       'TIEMPO_AFILIACION', 'CANCER_MAMA_FAMILIAR', 'CANCER_OTRO_SITIO',
       'CANCER_OTRO_SITIO_FAMILIAR', 'HIPERTENSION', 'HIPERTENSION_FAMILIAR',
       'DIABETES', 'DIABETES_FAMILIAR', 'CORONARIOS', 'CORONARIOS_FAMILIAR',
       'CEREBRAL', 'CEREBRAL_FAMILIAR', 'ENFERMEDAD_RENAL',
       'ENFERMEDAD_RENAL_FAMILIAR', 'OTROS_ANTECEDENTES_VASCULARES',
       'Pendiente', 'Intercepto', 'Promedio_costo'],
      dtype='object')

Formateamos los nombres de las variables

In [6]:
def format_columns(df):
    result = df.copy()
    new_cols = []
    for col in result.columns:
        new_cols.append(re.sub(r'\s+', ' ',col.strip()).replace(' ','_').lower())
    result.columns = new_cols
    return result

*Descripciones de variables:* El significado y tipo (e.g. nominal/ordinal/intervalo/ratio) de cada una de las variables
Ejemplo:
- GRE_Score **(Ratio - Discreta)**: Puntuación en el Graduate Record Examination (GRE), un examen estandarizado utilizado para la admisión a programas de posgrado cuyo rango es desde 260 hasta 340.
- TOEFL_Score **(Ratio - Discreta)**: Puntuación en el Test of English as a Foreign Language (TOEFL), un examen estandarizado para medir la habilidad en inglés de hablantes no nativos cuyo rango es desde 0 hasta 120.
- University_Rating **(Ordinal)**: Calificación de la universidad, generalmente en una escala de 1 a 5, donde 5 es la más alta.

Falta completar---->

**agrupacion_diastolica:** Resultado Ultima Toma Tension Sistolica

**agrupacion_sistolica:** Resultado Ultima Toma Tension Diastolica

**cancer_mama_familiar:** Tienen Antecedentes De Cancer_Mama_Familiar

**cancer_otro_sitio:** El Usuario Tiene Cancer_Otro_Sitio Fuente Antecedentes

**cancer_otro_sitio_familiar:** Tienen Antecedentes De Cancer_Otro_Sitio_Familiar

**cant_fliar_cp:** Tiene Familiar con Cancer de Prostata?

**cant_fliar_riesgos:** Cantidad de riesgos del grupo Familiar

**cant_gr_flia:** Cantidad de personas en el grupo familiar

**cant_riesgos_flia_mean:** Cantidad promedio de riesgos en familia

**cantidad_serv_flia:** Cantidad de Servicios prestados a la familia en el ultimo semestre

**cantidad_servicios:** Cantidad de Servicios prestados al usuario en el ultimo semestre

**cerebral:** Ha presentado enfermedad cerebral?

**cerebral_familiar:** Tienen Antecedentes De Cerebral_Familiar

**conteo_dx_diferentes:** Cuantos Diagnosticos ha presentado en el ultimo semestre

**coronarios:** Ha presentado enfermedad coronaria?

**coronarios_familiar:** Tienen Antecedentes De Coronarios Familiar

**diabetes:** Ha presentado Diabetes?

**diabetes_familiar:** Tienen Antecedentes De Diabetes Familiar

**edad:** Edad del Usuario

**enfermedad_renal:** Ha presentado Enfermedad Renal?

**enfermedad_renal_familiar:** Tienen Antecedentes De Enfermedad Renal Familiar

**estado_civil:** Estado Civil del Usuario

**estrato:** Estrato sociodemografico del usuario

**genero:** Genero Usuario

**grupo_etareo:** Grupo Etareo

**hipertension:** Ha presentado HTA?

**hipertension_familiar:** Tienen Antecedentes De HTA Familiar

**imc:** Indice de masa corporal

**intercepto:** Intercepto del costo del ultimo semestre

**intercepto_flia:** Incercepto del costo familiar del ultimo semetre

**medicamentos:** Cantidad de medicamentos usados en el ultimo semestre

**medicina_especializada:** Cantidad de servicios de medicina especializada en el ultimo semestre

**medicina_general:** Cantidad de servicios de medicina general en el ultimo semestre

**min_tiempo_cp_fliar:** Tiempo desde la marca del CA de Prostata Familiar

**otros_antecedentes_vasculares:** Tienen Antecedentes De enfermedad vascular?

**parentesco:** Parentezco con el cotizante

**pendiente:** Pendiente del costo en el ultimo semestre

**pendiente_flia:** Pendiente del costo familiar en el ultimo semestre

**perdida_de_peso:** Peso perdido/ganado en el ultimo año

**programa:** Programa actual del usuario

**promedio_costo:** Promedio del costo en el ultimo semestre

**promedio_costo_flia:** Promedio del costo familiar en el ultimo semestre

**psa_max_gr_flia:** Resultado PSA Maximo del grupo familiar

**psa_min_gr_flia:** Resultado PSA Minimo del grupo familiar

**riesgos:** cantidad de riesgos del usuario

**target:** Resultado numerico del PSA

**target_clase:** Resultado anormal o normal del PSA

**tiempo_afiliacion:** Tiempo de afiliación del usuario

**tiempo_ultima_cita:** Tiempo desde la ultima cita del usuario

In [7]:
# Creamos un Transformer para renombrar columnas
class Rename_columns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        result = X.copy()
        result = format_columns(result)
        result = result.rename(columns={'estado_civi':'estado_civil'})
        return result

In [14]:
# Preguntar si debo meter esto dentro del pipeline gigante de todo???
df_renamed = Rename_columns().fit_transform(df)
df1_renamed = Rename_columns().fit_transform(df1)

In [9]:
df_renamed.columns

Index(['medicamentos', 'medicina_especializada', 'medicina_general',
       'cant_gr_flia', 'cant_riesgos_flia_mean', 'min_tiempo_cp_fliar',
       'cant_fliar_cp', 'psa_max_gr_flia', 'psa_min_gr_flia',
       'cant_fliar_riesgos', 'cantidad_serv_flia', 'pendiente_flia',
       'intercepto_flia', 'promedio_costo_flia', 'estado_civil', 'programa',
       'estrato', 'imc', 'agrupacion_sistolica', 'agrupacion_diastolica',
       'parentesco', 'edad', 'riesgos', 'cantidad_servicios',
       'perdida_de_peso', 'tiempo_ultima_cita', 'conteo_dx_diferentes',
       'tiempo_afiliacion', 'cancer_mama_familiar', 'cancer_otro_sitio',
       'cancer_otro_sitio_familiar', 'hipertension', 'hipertension_familiar',
       'diabetes', 'diabetes_familiar', 'coronarios', 'coronarios_familiar',
       'cerebral', 'cerebral_familiar', 'enfermedad_renal',
       'enfermedad_renal_familiar', 'otros_antecedentes_vasculares', 'target',
       'pendiente', 'intercepto', 'promedio_costo'],
      dtype='object')

In [10]:
df1_renamed.columns

Index(['medicamentos', 'medicina_especializada', 'medicina_general',
       'cant_gr_flia', 'cant_riesgos_flia_mean', 'min_tiempo_cp_fliar',
       'cant_fliar_cp', 'psa_max_gr_flia', 'psa_min_gr_flia',
       'cant_fliar_riesgos', 'cantidad_serv_flia', 'pendiente_flia',
       'intercepto_flia', 'promedio_costo_flia', 'estado_civil', 'programa',
       'estrato', 'imc', 'agrupacion_sistolica', 'agrupacion_diastolica',
       'parentesco', 'edad', 'riesgos', 'cantidad_servicios',
       'perdida_de_peso', 'tiempo_ultima_cita', 'conteo_dx_diferentes',
       'tiempo_afiliacion', 'cancer_mama_familiar', 'cancer_otro_sitio',
       'cancer_otro_sitio_familiar', 'hipertension', 'hipertension_familiar',
       'diabetes', 'diabetes_familiar', 'coronarios', 'coronarios_familiar',
       'cerebral', 'cerebral_familiar', 'enfermedad_renal',
       'enfermedad_renal_familiar', 'otros_antecedentes_vasculares',
       'pendiente', 'intercepto', 'promedio_costo'],
      dtype='object')

## tonterias para probar

In [11]:
# Separar variable objetivo y características
X = df_renamed.drop('target', axis=1)
y = df_renamed['target']

# Separar en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocesamiento
# Definir las columnas numéricas y categóricas
numeric_features = df_renamed.select_dtypes(include=['int64', 'float64'])
categorical_features = df_renamed.select_dtypes(include=['object', 'category'])

# Pipeline de preprocesamiento
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Pipeline del modelo
model_svc = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=10)),  # Opcional, si hay muchas características
    ('classifier', SVC(probability=True))])

# Grid Search para optimización de hiperparámetros en SVC
param_grid = {
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__C': [0.1, 1, 10],
    'classifier__gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(model_svc, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)

# Evaluación del mejor modelo
best_model = grid_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'Mejor modelo: {grid_search.best_params_}')
print(f'ROC_AUC: {roc_auc}')

# Curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)


ValueError: 
All the 60 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\alfa7\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\alfa7\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\alfa7\AppData\Roaming\Python\Python312\site-packages\sklearn\pipeline.py", line 471, in fit
    Xt = self._fit(X, y, routed_params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\alfa7\AppData\Roaming\Python\Python312\sit

In [12]:
numeric_features

,medicamentos,medicina_especializada,medicina_general,cant_gr_flia,cant_riesgos_flia_mean,min_tiempo_cp_fliar,cant_fliar_cp,psa_max_gr_flia,psa_min_gr_flia,cant_fliar_riesgos,...,edad,riesgos,cantidad_servicios,perdida_de_peso,tiempo_ultima_cita,conteo_dx_diferentes,tiempo_afiliacion,pendiente,intercepto,promedio_costo
ID,,,,,,,,,,,,,,,,,,,,,
16484,12.0,3.0,3.0,2,0.0,NaN,0.0,NaN,NaN,0.0,...,55.0,NaN,14.0,NaN,1.0,5.0,310.0,58373.057143,-110021.200000,1.885690e+05
11528,6.0,1.0,1.0,2,0.5,NaN,0.0,NaN,NaN,1.0,...,55.0,NaN,9.0,NaN,1.0,3.0,21.0,19618.942857,-36176.133333,9.747050e+04
26630,1.0,0.0,2.0,2,0.0,NaN,0.0,NaN,NaN,0.0,...,59.0,NaN,17.0,NaN,1.0,5.0,280.0,398534.000000,-626628.000000,2.304723e+06
12586,6.0,0.0,2.0,2,0.0,NaN,0.0,NaN,NaN,0.0,...,55.0,NaN,4.0,NaN,1.0,3.0,65.0,1292.600000,13410.733333,5.380450e+04
11225,3.0,0.0,2.0,1,1.0,NaN,0.0,NaN,NaN,1.0,...,53.0,1.0,6.0,NaN,1.0,3.0,28.0,13728.714286,-27375.666667,4.134967e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7141,0.0,0.0,1.0,3,0.0,NaN,0.0,NaN,NaN,0.0,...,55.0,NaN,2.0,NaN,1.0,1.0,62.0,6628.571429,-15466.666667,4.640000e+04
15417,6.0,0.0,3.0,1,1.0,NaN,0.0,NaN,NaN,1.0,...,49.0,1.0,7.0,-6.25,1.0,3.0,21.0,36161.142857,-66307.333333,9.038500e+04
19389,8.0,2.0,2.0,1,0.0,NaN,0.0,NaN,NaN,0.0,...,53.0,NaN,11.0,1.35,1.0,5.0,31.0,15912.885714,16190.066667,1.078278e+05


In [13]:
categorical_features

,estado_civil,programa,estrato,imc,agrupacion_sistolica,agrupacion_diastolica,parentesco,cancer_mama_familiar,cancer_otro_sitio,cancer_otro_sitio_familiar,...,hipertension_familiar,diabetes,diabetes_familiar,coronarios,coronarios_familiar,cerebral,cerebral_familiar,enfermedad_renal,enfermedad_renal_familiar,otros_antecedentes_vasculares
ID,,,,,,,,,,,,,,,,,,,,,
16484,UL,POS,1,Sobrepeso,Normal,Normal,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11528,SO,POS,1,Sobrepeso,Normal,Normal,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26630,CA,POS,1,Sobrepeso,Normal,Normal,None,0,0,0,...,1,0,0,0,0,0,0,0,0,0
12586,UL,POS,1,Sobrepeso,Normal,Normal,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11225,SO,POS,1,Obesidad,Normal,Normal,None,None,2,2,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7141,UL,POS,1,Bajo peso,Normal,Normal,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15417,SO,POS,1,Sobrepeso,Normal,Normal,None,None,2,2,...,2,2,2,2,2,2,2,2,2,2
19389,SO,POS,1,Sobrepeso,Normal,Hipotenso,None,None,2,2,...,2,2,2,2,2,2,2,2,2,2
